<a href="https://colab.research.google.com/github/Kryptera-K/ACN-Aroon-Laguerre-Adaptive-Volatility-Strategy/blob/main/ACN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install vectorbt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.7/527.7 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 18.5 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import vectorbt as vbt

# -------------------------
# Download Data
# -------------------------

symbol = "ACN"
start_date = "2000-01-01"
end_date = "2026-01-01"
interval = "1d"

df = yf.download(symbol, start=start_date, end=end_date, interval=interval, multi_level_index=False)
df.to_csv("ACN_clean.csv", index=False)
df

/tmp/ipython-input-1124259308.py:15: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(symbol, start=start_date, end=end_date, interval=interval, multi_level_index=False)
[*********************100%***********************]  1 of 1 completed


,Close,High,Low,Open,Volume
Date,,,,,
2001-07-19,10.603370,10.687246,10.484545,10.554442,34994300
2001-07-20,10.491530,10.519489,10.344747,10.519489,9238500
2001-07-23,10.484540,10.491530,10.170004,10.484540,7501000
2001-07-24,10.386686,10.463573,10.274851,10.449594,3537300
2001-07-25,10.449593,10.449593,10.239902,10.274850,4208100
...,...,...,...,...,...
2025-10-17,238.389999,239.119995,233.600006,234.350006,4436100
2025-10-20,242.179993,244.190002,239.740005,240.100006,3495200
2025-10-21,250.509995,252.089996,242.520004,242.520004,4475500


In [ ]:
# -------------------------
# Necessary Parameters
# -------------------------

AROON_PERIOD = 14
ATR_LEVEL = 2.0
ATR_PERIOD = 14
ATR_SHIFT = 5
DEM_LEVEL = 0.5
DEM_PERIOD = 14
DEM_SHIFT_1 = 5
DEM_SHIFT_2 = 10
DEM_SHIFT_3 = 15
LRSI_GAMMA = 0.5
LRSI_LEVEL = 0.5

# -------------------------
# Indicator Functions
# -------------------------

def atr_change_down(df, period=ATR_PERIOD):
    df = calculate_atr(df, period)
    return df['ATR'] < df['ATR'].shift(1)


def calculate_atr(df, period=ATR_PERIOD):
    """
    Calculate ATR (Average True Range)
    df : pandas DataFrame with columns ['High', 'Low', 'Close']
    period : ATR period (default 14)
    """
    df = df.copy()
    high_low = df['High'] - df['Low']
    high_close = abs(df['High'] - df['Close'].shift(1))
    low_close = abs(df['Low'] - df['Close'].shift(1))

    tr = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
    df['ATR'] = tr.rolling(window=period, min_periods=1).mean()
    return df


def demarker_is_lower_than_level(df, level=DEM_LEVEL):
    df = calculate_demarker(df)
    return df['DeMarker'] < level


def calculate_demarker(df, period=DEM_PERIOD):
    """
    Calculate DeMarker indicator.
    """
    df = df.copy()

    # DeMax and DeMin
    df['DeMax'] = np.where(df['High'] > df['High'].shift(1),
                           df['High'] - df['High'].shift(1), 0)
    df['DeMin'] = np.where(df['Low'] < df['Low'].shift(1),
                           df['Low'].shift(1) - df['Low'], 0)

    # Rolling sums
    dem_max = df['DeMax'].rolling(window=period).sum()
    dem_min = df['DeMin'].rolling(window=period).sum()

    df['DeMarker'] = dem_max / (dem_max + dem_min)
    return df


def laguerre_rsi_upward(df, gamma=LRSI_GAMMA):
    df = df.copy()
    df['LRsi'] = calculate_laguerre_rsi(df['Close'], gamma)
    return df['LRsi'].diff() > 0


def calculate_laguerre_rsi(series, gamma=LRSI_GAMMA):
    """
    Calculate Laguerre RSI for a pandas Series of close prices.
    Returns a pandas Series of Laguerre RSI values.
    """
    L0 = L1 = L2 = L3 = 0
    lrsi = []

    for price in series:
        L0 = (1 - gamma) * price + gamma * L0
        L1 = -gamma * L0 + L0 + gamma * L1
        L2 = -gamma * L1 + L1 + gamma * L2
        L3 = -gamma * L2 + L2 + gamma * L3

        CU = max(L0 - L1, 0) + max(L1 - L2, 0) + max(L2 - L3, 0)
        CD = max(L1 - L0, 0) + max(L2 - L1, 0) + max(L3 - L2, 0)

        lrsi.append(CU / (CU + CD) if (CU + CD) != 0 else 0)

    return pd.Series(lrsi, index=series.index)


def aroon_up_rises_from_bottom(df, period=AROON_PERIOD):
    df = calculate_aroon(df, period)
    return (df['Aroon_Up'].shift(1) < 30) & (df['Aroon_Up'] > df['Aroon_Up'].shift(1))


def calculate_aroon(df, period=AROON_PERIOD):
    """
    Vectorized calculation of Aroon Up and Aroon Down indicators.
    """
    high_array = df['High'].to_numpy()
    low_array = df['Low'].to_numpy()

    # Prepare arrays to store results
    aroon_up = np.full(len(df), np.nan)
    aroon_down = np.full(len(df), np.nan)

    for i in range(period - 1, len(df)):
        high_window = high_array[i - period + 1:i + 1]
        low_window = low_array[i - period + 1:i + 1]

        # Days since highest high / lowest low
        days_since_high = period - 1 - high_window[::-1].argmax()
        days_since_low = period - 1 - low_window[::-1].argmin()

        # Calculate Aroon values
        aroon_up[i] = (period - days_since_high) / period * 100
        aroon_down[i] = (period - days_since_low) / period * 100

    df['Aroon_Up'] = aroon_up
    df['Aroon_Down'] = aroon_down

    return df



# -------------------------
# Entry conditions
# -------------------------

df["ATR_Downward"] = atr_change_down(df)
df["Demarker_Is_Lower_Than_Level"] = demarker_is_lower_than_level(df)

# -------------------------
# Exit conditions
# -------------------------

df["LRsi_Upward"] = laguerre_rsi_upward(df)
df["Aroon_Up_Rises_From_Bottom"] = aroon_up_rises_from_bottom(df)

# -------------------------
# Signals
# -------------------------

entry_conditions = [
    'ATR_Downward',
    'Demarker_Is_Lower_Than_Level',
]
exit_conditions = [
    'LRsi_Upward',
    'Aroon_Up_Rises_From_Bottom',
]

df['entry_signal'] = df[entry_conditions].all(axis=1)
df['exit_signal']  = df[exit_conditions].all(axis=1)

# -------------------------
# Backtest
# -------------------------


shift_entries = df['entry_signal'].shift(1).astype(bool).fillna(False).to_numpy()
shift_exits = df['exit_signal'].shift(1).astype(bool).fillna(False).to_numpy()

pf = vbt.Portfolio.from_signals(
    close=df['Open'],
    entries=shift_entries,
    exits=shift_exits,
    init_cash=100_000,
    fees=0.001,
    slippage=0.002,
    freq='1d'
)


# -------------------------
# Portfolio Stats / Plot
# -------------------------

print(pf.stats())
pf.plot().show()

Start                                2001-07-19 00:00:00
End                                  2025-10-23 00:00:00
Period                                6103 days 00:00:00
Start Value                                     100000.0
End Value                                 3299639.790744
Total Return [%]                             3199.639791
Benchmark Return [%]                         2231.340647
Max Gross Exposure [%]                             100.0
Total Fees Paid                            198950.078004
Max Drawdown [%]                               61.244573
Max Drawdown Duration                  647 days 00:00:00
Total Trades                                          77
Total Closed Trades                                   76
Total Open Trades                                      1
Open Trade PnL                           -1508929.160201
Win Rate [%]                                   73.684211
Best Trade [%]                                 40.188864
Worst Trade [%]                

In [ ]:
# Buy and Hold Performance Metrics
df_holding = df['Open']
pf_holding = vbt.Portfolio.from_holding(df_holding, init_cash=100_000 , freq='D')
print(pf_holding.stats())

Start                         2001-07-19 00:00:00
End                           2025-10-23 00:00:00
Period                         6103 days 00:00:00
Start Value                              100000.0
End Value                          2331340.646767
Total Return [%]                      2231.340647
Benchmark Return [%]                  2231.340647
Max Gross Exposure [%]                      100.0
Total Fees Paid                               0.0
Max Drawdown [%]                        62.251647
Max Drawdown Duration           964 days 00:00:00
Total Trades                                    1
Total Closed Trades                             0
Total Open Trades                               1
Open Trade PnL                     2231340.646767
Win Rate [%]                                  NaN
Best Trade [%]                                NaN
Worst Trade [%]                               NaN
Avg Winning Trade [%]                         NaN
Avg Losing Trade [%]                          NaN


In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# -------------------------
# Download Data
# -------------------------
symbol = "ACN"
start_date = "2000-01-01"
end_date = "2026-01-01"
interval = "1d"

df = yf.download(symbol, start=start_date, end=end_date, interval=interval, multi_level_index=False)

# -------------------------
# Parameters
# -------------------------
AROON_PERIOD = 14
ATR_PERIOD = 14
DEM_PERIOD = 14
LRSI_GAMMA = 0.5

# -------------------------
# Indicator Functions
# -------------------------
def calculate_atr(df, period=ATR_PERIOD):
    df = df.copy()
    high_low = df['High'] - df['Low']
    high_close = abs(df['High'] - df['Close'].shift(1))
    low_close = abs(df['Low'] - df['Close'].shift(1))
    tr = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
    df['ATR'] = tr.rolling(window=period, min_periods=1).mean()
    return df

def calculate_demarker(df, period=DEM_PERIOD):
    df = df.copy()
    df['DeMax'] = np.where(df['High'] > df['High'].shift(1),
                           df['High'] - df['High'].shift(1), 0)
    df['DeMin'] = np.where(df['Low'] < df['Low'].shift(1),
                           df['Low'].shift(1) - df['Low'], 0)
    dem_max = df['DeMax'].rolling(window=period).sum()
    dem_min = df['DeMin'].rolling(window=period).sum()
    df['DeMarker'] = dem_max / (dem_max + dem_min)
    return df

def calculate_laguerre_rsi(series, gamma=LRSI_GAMMA):
    L0 = L1 = L2 = L3 = 0
    lrsi = []
    for price in series:
        L0 = (1 - gamma) * price + gamma * L0
        L1 = -gamma * L0 + L0 + gamma * L1
        L2 = -gamma * L1 + L1 + gamma * L2
        L3 = -gamma * L2 + L2 + gamma * L3
        CU = max(L0 - L1, 0) + max(L1 - L2, 0) + max(L2 - L3, 0)
        CD = max(L1 - L0, 0) + max(L2 - L1, 0) + max(L3 - L2, 0)
        lrsi.append(CU / (CU + CD) if (CU + CD) != 0 else 0)
    return pd.Series(lrsi, index=series.index)

def calculate_aroon(df, period=AROON_PERIOD):
    high_array = df['High'].to_numpy()
    low_array = df['Low'].to_numpy()
    aroon_up = np.full(len(df), np.nan)
    aroon_down = np.full(len(df), np.nan)
    for i in range(period - 1, len(df)):
        high_window = high_array[i - period + 1:i + 1]
        low_window = low_array[i - period + 1:i + 1]
        days_since_high = period - 1 - high_window[::-1].argmax()
        days_since_low = period - 1 - low_window[::-1].argmin()
        aroon_up[i] = (period - days_since_high) / period * 100
        aroon_down[i] = (period - days_since_low) / period * 100
    df['Aroon_Up'] = aroon_up
    df['Aroon_Down'] = aroon_down
    return df

# -------------------------
# Calculate Indicators
# -------------------------
df = calculate_atr(df)
df = calculate_demarker(df)
df['LRsi'] = calculate_laguerre_rsi(df['Close'])
df = calculate_aroon(df)

# -------------------------
# Plot with Plotly
# -------------------------
fig = make_subplots(
    rows=5, cols=1, shared_xaxes=True, vertical_spacing=0.02,
    row_heights=[0.4, 0.15, 0.15, 0.15, 0.15],
    subplot_titles=("Price", "ATR", "DeMarker", "Laguerre RSI", "Aroon Up/Down")
)

# --- Price Chart ---
fig.add_trace(go.Candlestick(
    x=df.index, open=df['Open'], high=df['High'], low=df['Low'], close=df['Close'],
    name='Candlestick'), row=1, col=1)

# --- ATR ---
fig.add_trace(go.Scatter(x=df.index, y=df['ATR'], mode='lines', name='ATR'), row=2, col=1)

# --- DeMarker ---
fig.add_trace(go.Scatter(x=df.index, y=df['DeMarker'], mode='lines', name='DeMarker', line=dict(color='orange')), row=3, col=1)
fig.add_hline(y=0.7, line_dash="dot", line_color="red", row=3, col=1)
fig.add_hline(y=0.3, line_dash="dot", line_color="green", row=3, col=1)

# --- Laguerre RSI ---
fig.add_trace(go.Scatter(x=df.index, y=df['LRsi'], mode='lines', name='Laguerre RSI', line=dict(color='purple')), row=4, col=1)
fig.add_hline(y=0.8, line_dash="dot", line_color="red", row=4, col=1)
fig.add_hline(y=0.2, line_dash="dot", line_color="green", row=4, col=1)

# --- Aroon ---
fig.add_trace(go.Scatter(x=df.index, y=df['Aroon_Up'], mode='lines', name='Aroon Up', line=dict(color='blue')), row=5, col=1)
fig.add_trace(go.Scatter(x=df.index, y=df['Aroon_Down'], mode='lines', name='Aroon Down', line=dict(color='red')), row=5, col=1)

# -------------------------
# Layout Settings
# -------------------------
fig.update_layout(
    title=f"Technical Indicators for {symbol}",
    xaxis_rangeslider_visible=False,
    height=1200,
    showlegend=True,
    template="plotly_white"
)

fig.show()


/tmp/ipython-input-625389974.py:15: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  1 of 1 completed
